In [106]:
import pandas as pd
import numpy as np

diabetes= pd.read_csv('data/diabetes_data.csv', sep=',')
diabetes.head()
#Признаки
# Pregnancies — количество беременностей.
# Glucose — концентрация глюкозы в плазме через два часа при пероральном тесте на толерантность к глюкозе.
# BloodPressure — диастолическое артериальное давление (мм рт. ст.).
# SkinThickness — толщина кожной складки трицепса (мм).
# Insulin — двухчасовой сывороточный инсулин (ме Ед/мл).
# BMI — индекс массы тела (весвкг/роствм).
# DiabetesPedigreeFunction — функция родословной диабета (чем она выше, тем выше шанс наследственной заболеваемости).
# Age — возраст.
# Outcome — наличие диабета (0 — нет, 1 — да).
diabetes.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 778 entries, 0 to 777
Data columns (total 10 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Pregnancies               778 non-null    int64  
 1   Glucose                   778 non-null    int64  
 2   BloodPressure             778 non-null    int64  
 3   SkinThickness             778 non-null    int64  
 4   Insulin                   778 non-null    int64  
 5   BMI                       778 non-null    float64
 6   DiabetesPedigreeFunction  778 non-null    float64
 7   Age                       778 non-null    int64  
 8   Outcome                   778 non-null    int64  
 9   Gender                    778 non-null    object 
dtypes: float64(2), int64(7), object(1)
memory usage: 60.9+ KB


In [107]:
diabetes.describe()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
count,778.000000,778.000000,778.000000,778.000000,778.000000,778.000000,778.000000,778.000000,778.000000
mean,3.848329,120.822622,69.035990,20.457584,79.521851,31.982262,0.470871,33.317481,0.344473
std,3.360782,31.883264,19.432323,15.954452,114.862405,7.853917,0.330669,11.816300,0.475502
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.078000,21.000000,0.000000
25%,1.000000,99.000000,62.000000,0.000000,0.000000,27.300000,0.241500,24.000000,0.000000
50%,3.000000,117.000000,72.000000,23.000000,27.000000,32.000000,0.370000,29.000000,0.000000
75%,6.000000,140.000000,80.000000,32.000000,126.750000,36.500000,0.625500,41.000000,1.000000
max,17.000000,199.000000,122.000000,99.000000,846.000000,67.100000,2.420000,81.000000,1.000000


In [108]:
duplicates = diabetes[diabetes.duplicated()]
print(f'Число дублткатов: {duplicates.shape[0]}')
diabetes = diabetes.drop_duplicates()
print(f'Результирующее число записей: {diabetes.shape[0]}')



Число дублткатов: 10
Результирующее число записей: 768


In [109]:
# Далее найдите все неинформативные признаки в данных и избавьтесь от них. В качестве порога информативности возьмите 0.95: 
# удалите все признаки, для которых 95 % значений повторяются или 95 % записей уникальны. В ответ запишите имена признаков, которые вы нашли (без кавычек).

low_information_cols = []
for col in diabetes.columns:
    top_freq = diabetes[col].value_counts(normalize=True).max()
    nunique_ratio = diabetes[col].nunique() / diabetes[col].count()
    if top_freq > 0.95:
        low_information_cols.append(col)
        print(f'{col}:{round(top_freq * 100,2)}% одинаковых значений')
    if nunique_ratio > 0.95:
        low_information_cols.append(col)
        print(f'{col}:{round(nunique_ratio * 100,2)}% уникальных значений')   
        
diabetes = diabetes.drop(low_information_cols, axis=1)         

Gender:100.0% одинаковых значений


In [110]:
#Замените все записи, равные 0, в столбцах Glucose, BloodPressure, SkinThickness, Insulin и BMI на символ пропуска. 
# Его вы можете взять из библиотеки numpy: np.nan.
# Подсказка (1 из 1): Создайте функцию, которая возвращает np.nan, если её аргумент равен 0, в противном случае — возвращает аргумент. 
# Воспользуйтесь методом apply() и примените созданную функцию для каждого перечисленного столбца.

def nan_function(x):
     return np.nan if x == 0 else x

diabetes["Glucose"] = diabetes["Glucose"].apply(nan_function)
diabetes["BloodPressure"] = diabetes["BloodPressure"].apply(nan_function)
diabetes["SkinThickness"] = diabetes["SkinThickness"].apply(nan_function)
diabetes["Insulin"] = diabetes["Insulin"].apply(nan_function)
diabetes["BMI"] = diabetes["BMI"].apply(nan_function)
# diabetes.replace(np.nan, 0)
diabetes.isnull().mean().round(2).sort_values(ascending=False)

Insulin                     0.49
SkinThickness               0.30
BloodPressure               0.05
Glucose                     0.01
BMI                         0.01
Pregnancies                 0.00
DiabetesPedigreeFunction    0.00
Age                         0.00
Outcome                     0.00
dtype: float64

In [111]:
# Удалите из данных признаки, где число пропусков составляет более 30 %. 
# Сколько признаков осталось в ваших данных (с учетом удаленных неинформативных признаков в задании 8.2)?
thresh = diabetes.shape[0]*0.7
diabetes = diabetes.dropna(thresh=thresh, axis=1)
print(diabetes.shape[1])

8


In [112]:
#Удалите из данных только те строки, в которых содержится более двух пропусков одновременно. 
# Чему равно результирующее число записей в таблице?
m = diabetes.shape[1]
diabetes = diabetes.dropna(thresh= m - 2, axis=0)
diabetes.shape[0]
diabetes.isnull().mean().round(2).sort_values(ascending=False)


,Pregnancies,Glucose,BloodPressure,SkinThickness,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,98.0,58.0,33.0,34.0,0.430,43,0
1,2,112.0,75.0,32.0,35.7,0.148,21,0
2,2,108.0,64.0,NaN,30.8,0.158,21,0
3,8,107.0,80.0,NaN,24.6,0.856,34,0
4,7,136.0,90.0,NaN,29.9,0.210,50,0
...,...,...,...,...,...,...,...,...
763,5,139.0,64.0,35.0,28.6,0.411,26,0
764,1,96.0,122.0,NaN,22.4,0.207,27,0
765,10,101.0,86.0,37.0,45.6,1.136,38,1
766,0,141.0,NaN,NaN,42.4,0.205,29,1


In [123]:
# В оставшихся записях замените пропуски на медиану.
# Чему равно среднее значение в столбце SkinThickness? Ответ округлите до десятых
# values = {'SkinThickness':diabetes['SkinThickness'].median(),
#           'BloodPressure':diabetes['BloodPressure'].median(),
#           'Glucose':diabetes['Glucose'].median(),
#           'BMI':diabetes['BMI'].median()}
# diabetes = diabetes.fillna(values)
# print(round(diabetes['SkinThickness'].mean(), 1))

#Возможный вариант решения: 
null_data = diabetes.isnull().sum()
cols = null_data[null_data > 0].index
for col in cols:
  diabetes[col] = diabetes[col].fillna(diabetes[col].median())
  
print(diabetes['SkinThickness'].mean().round(1))


29.1


In [125]:
#Сколько выбросов найдёт классический метод межквартильного размаха в признаке SkinThickness?
def outliers_iqr(data, feature):
    x = data[feature]
    quartile_1, quartile_3 = x.quantile(0.25), x.quantile(0.75)
    iqr = quartile_3 - quartile_1
    lower_bound = quartile_1 - (iqr * 1.5)
    upper_bound = quartile_3 + (iqr * 1.5)
    outliers = data[(x < lower_bound) | (x > upper_bound)]
    cleaned = data[(x >= lower_bound) & (x <= upper_bound)]
    return outliers, cleaned

outliers, cleaned = outliers_iqr(diabetes, 'SkinThickness')
print(f'число выбросов: {outliers.shape[0]}')
#Возможный вариант решения:

# def outliers_iqr_mod(data, feature, left=1.5, right=1.5, log_scale=False):
#     if log_scale:
#         x = np.log(data[feature])
#     else:
#         x= data[feature]
#     quartile_1, quartile_3 = x.quantile(0.25), x.quantile(0.75),
#     iqr = quartile_3 - quartile_1
#     lower_bound = quartile_1 - (iqr * left)
#     upper_bound = quartile_3 + (iqr * right)
#     outliers = data[(x < lower_bound) | (x > upper_bound)]
#     cleaned = data[(x >= lower_bound) & (x <= upper_bound)]
#     return outliers, cleaned
# outliers, _ = outliers_iqr_mod(diabetes, 'SkinThickness')
# print(outliers.shape[0])


число выбросов: 87


In [127]:
#Сколько выбросов найдёт классический метод z-отклонения в признаке SkinThickness?
def outliers_z_mod(data, feature, left = 3 , right = 3, log_scale = False):
    if log_scale:
        x = np.log(data[feature] + 1)
    else:
        x = data[feature]
    mu = x.mean()
    sigma = x.std()
    lower_bound = mu - (left * sigma)
    upper_bound = mu + (right * sigma)
    outliers = data[(x < lower_bound) | (x > upper_bound)]
    cleaned = data[(x >= lower_bound) & (x <= upper_bound)]
    return outliers, cleaned

outliers, cleaned = outliers_z_mod(diabetes, 'SkinThickness')
print(f'число выбросов: {outliers.shape[0]}')
            

число выбросов: 4


In [135]:
# На приведённой гистограмме показано распределение признака DiabetesPedigreeFunction.
# Такой вид распределения очень похож на логнормальный, и он заставляет задуматься о логарифмировании признака. 
# Найдите сначала число выбросов в признаке DiabetesPedigreeFunction с помощью классического метода межквартильного размаха.
# Затем найдите число выбросов в этом же признаке в логарифмическом масштабе (при логарифмировании единицу прибавлять не нужно!).
# Какова разница между двумя этими числами (вычтите из первого второе)?

def outliers_iqr_mod(data, feature, left=1.5, right=1.5, log_scale=False):
    if log_scale:
        x = np.log(data[feature])
    else:
        x= data[feature]
    quartile_1, quartile_3 = x.quantile(0.25), x.quantile(0.75),
    iqr = quartile_3 - quartile_1
    lower_bound = quartile_1 - (iqr * left)
    upper_bound = quartile_3 + (iqr * right)
    outliers = data[(x < lower_bound) | (x > upper_bound)]
    cleaned = data[(x >= lower_bound) & (x <= upper_bound)]
    return outliers, cleaned
outliers_classic, _ = outliers_iqr_mod(diabetes, 'DiabetesPedigreeFunction')
print(outliers_classic.shape[0])
outliers_log, _ = outliers_iqr_mod(diabetes, 'DiabetesPedigreeFunction', log_scale= True)
print(outliers_log.shape[0])

29
0
